In [1]:
import os

# move to project root
while True:
    # get list of directories
    dirs = os.listdir()
    if "README.md" in dirs:
        break
    else:
        os.chdir("..")
print(os.getcwd())

/mnt/antares_raid/home/bramantyos/codes/multilang_timescale


In [2]:
import json

from typing import List
import h5py


from itertools import product

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/tmp/ipykernel_49860/4119703018.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from src.trainer import Trainer
from src.settings import TrainerConfig, SubjectConfig, FeatureConfig, ResultConfig

from src.plotting import plot_flatmap_from_mapper, plot_timeline_flatmaps

In [4]:
from himalaya import kernel_ridge

In [5]:
result_meta_dir = ".temp/result_meta"

In [6]:
# scanning result meta json files and put it into a dataframe
result_meta_files = os.listdir(result_meta_dir)
result_meta_files = [f for f in result_meta_files if f.endswith(".json")]
## read json and cast it into ResultConfig
result_meta_list = []
for f in result_meta_files:
    with open(os.path.join(result_meta_dir, f), "r") as f:
        result_config = ResultConfig(**json.load(f))
        result_meta_list.append(result_config.dict())

result_meta_df = pd.DataFrame(result_meta_list)

# add result_meta_files to result_meta_df
result_meta_df["result_meta_file"] = [
    os.path.join(result_meta_dir, f) for f in result_meta_files
]

In [7]:
result_meta_df.head()

,subject_config_path,feature_config_path,trainer_config_path,result_dir,hyperparam_path,stats_path,plot_dir,result_meta_file
0,.temp/config/subject/subject-07-reading.json,.temp/config/feature/mBERT-all_timescales-7-fe...,.temp/config/train/trainer_config.json,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05ddb,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result_meta/637f0762-f0b7-4bb0-878d-08ff...
1,.temp/config/subject/subject-07-reading.json,.temp/config/feature/mBERT-all_timescales-7-fe...,.temp/config/train/trainer_config_shorttime.json,.temp/result/cfdc6501-ad95-4992-9c8a-57a705ac6a86,.temp/result/cfdc6501-ad95-4992-9c8a-57a705ac6...,.temp/result/cfdc6501-ad95-4992-9c8a-57a705ac6...,.temp/result/cfdc6501-ad95-4992-9c8a-57a705ac6...,.temp/result_meta/cfdc6501-ad95-4992-9c8a-57a7...
2,.temp/config/subject/subject-07-listening.json,.temp/config/feature/BERT-all_timescales-7-fea...,.temp/config/train/trainer_config.json,.temp/result/80671ed6-4ee0-4d1c-88f1-095a8268e522,.temp/result/80671ed6-4ee0-4d1c-88f1-095a8268e...,.temp/result/80671ed6-4ee0-4d1c-88f1-095a8268e...,.temp/result/80671ed6-4ee0-4d1c-88f1-095a8268e...,.temp/result_meta/80671ed6-4ee0-4d1c-88f1-095a...
3,.temp/config/subject/subject-07-listening.json,.temp/config/feature/mBERT-all_timescales-6-fe...,.temp/config/train/trainer_config.json,.temp/result/c33c8b46-ad79-4c41-ba5c-dda2a242ebd2,.temp/result/c33c8b46-ad79-4c41-ba5c-dda2a242e...,.temp/result/c33c8b46-ad79-4c41-ba5c-dda2a242e...,.temp/result/c33c8b46-ad79-4c41-ba5c-dda2a242e...,.temp/result_meta/c33c8b46-ad79-4c41-ba5c-dda2...
4,.temp/config/subject/subject-07-reading.json,.temp/config/feature/mBERT-all_timescales-6-fe...,.temp/config/train/trainer_config.json,.temp/result/7eba7e23-9514-45a6-954b-d16617d106d2,.temp/result/7eba7e23-9514-45a6-954b-d16617d10...,.temp/result/7eba7e23-9514-45a6-954b-d16617d10...,.temp/result/7eba7e23-9514-45a6-954b-d16617d10...,.temp/result_meta/7eba7e23-9514-45a6-954b-d166...


In [8]:
result_meta_df["trainer_config_path"].unique()

array(['.temp/config/train/trainer_config.json',
       '.temp/config/train/trainer_config_shorttime.json'], dtype=object)

In [9]:
result_meta_df["subject_config_path"].unique()

array(['.temp/config/subject/subject-07-reading.json',
       '.temp/config/subject/subject-07-listening.json'], dtype=object)

In [10]:
result_meta_df["feature_config_path"].unique()

array(['.temp/config/feature/mBERT-all_timescales-7-feature_config.json',
       '.temp/config/feature/BERT-all_timescales-7-feature_config.json',
       '.temp/config/feature/mBERT-all_timescales-6-feature_config.json',
       '.temp/config/feature/BERT-all_timescales-6-feature_config.json'],
      dtype=object)

In [11]:
# select some results
selected_train_config = [".temp/config/train/trainer_config.json"]
selected_feature_config = [
    ".temp/config/feature/BERT-all_timescales-7-feature_config.json",
    ".temp/config/feature/mBERT-all_timescales-7-feature_config.json",
]
selected_subject_config = [".temp/config/subject/subject-07-reading.json"]

# now select dataframe
selected_result_meta_df = result_meta_df.query(
    "trainer_config_path in @selected_train_config and \
                                                feature_config_path in @selected_feature_config and \
                                                subject_config_path in @selected_subject_config"
)

In [12]:
selected_result_meta_df

,subject_config_path,feature_config_path,trainer_config_path,result_dir,hyperparam_path,stats_path,plot_dir,result_meta_file
0,.temp/config/subject/subject-07-reading.json,.temp/config/feature/mBERT-all_timescales-7-fe...,.temp/config/train/trainer_config.json,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05ddb,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result/637f0762-f0b7-4bb0-878d-08fff7f05...,.temp/result_meta/637f0762-f0b7-4bb0-878d-08ff...
6,.temp/config/subject/subject-07-reading.json,.temp/config/feature/BERT-all_timescales-7-fea...,.temp/config/train/trainer_config.json,.temp/result/5d9440b5-83fd-45b9-b23a-2973e00515fe,.temp/result/5d9440b5-83fd-45b9-b23a-2973e0051...,.temp/result/5d9440b5-83fd-45b9-b23a-2973e0051...,.temp/result/5d9440b5-83fd-45b9-b23a-2973e0051...,.temp/result_meta/5d9440b5-83fd-45b9-b23a-2973...


In [13]:
result_config_json_mbert = selected_result_meta_df.iloc[1].to_dict()
result_config_json_bert = selected_result_meta_df.iloc[0].to_dict()

In [14]:
bert_trainer = Trainer(result_config_json=result_config_json_bert["result_meta_file"])

In [15]:
bert_prediction = bert_trainer.refit_and_evaluate(return_pred=True)

refitting...
predicting in batches...
computing scores...
computing permutation test...
computing timescale selectivity...
saving stat...


/mnt/antares_raid/home/bramantyos/codes/multilang_timescale/src/utils.py:144: RuntimeWarning: invalid value encountered in divide
  normalized_scores = np.nan_to_num(nz_scores / score_sum)


In [42]:
from himalaya.scoring import r2_score_split
import multiprocessing

In [116]:
bert_test_pred = bert_prediction["test_pred"]
bert_test_targets = bert_trainer.test_data[:, bert_trainer.mask]

predictions = bert_test_pred
targets = bert_test_targets

score_func = r2_score_split

num_permutations = 50
permutation_block_size = 10
num_processes = 10


true_scores = score_func(targets, predictions)

num_TRs = predictions.shape[0]
blocks = np.array_split(np.arange(num_TRs), int(num_TRs / permutation_block_size))

 def single_test(repeats: int = 0, seed: int = 0):
    np.random.seed(seed)
    num_get_true_score = np.zeros(true_scores.shape)
    for i in range(repeats):
        np.random.shuffle(blocks)
        permutation_order = np.concatenate(blocks)
        shuffled_pred = predictions[permutation_order]
        shuffled_scores = score_func(targets, shuffled_pred)
        num_get_true_score[shuffled_scores >= true_scores] += 1
    return num_get_true_score

repeats = num_permutations // num_processes
seeds = np.random.randint(0, 1000000, num_processes)    

with multiprocessing.Pool(num_processes) as pool:
    num_get_true_scores = pool.starmap(single_test, zip([repeats] * num_processes, seeds))


In [114]:
num_get_true_score_sum = np.sum(num_get_true_scores, axis=0)

In [115]:
num_get_true_score_sum / (repeats * num_processes)

array([[0.6 , 0.88, 0.94, ..., 0.26, 1.  , 1.  ],
       [0.18, 0.44, 0.42, ..., 0.06, 0.58, 0.76],
       [0.64, 0.42, 0.2 , ..., 1.  , 0.9 , 0.28],
       ...,
       [0.44, 0.18, 0.72, ..., 0.6 , 0.44, 0.72],
       [0.78, 0.12, 0.6 , ..., 0.8 , 0.82, 0.84],
       [0.14, 0.36, 0.8 , ..., 0.94, 0.68, 0.74]])

In [19]:
# permutation test
def permutation_test(
    targets: np.ndarray,
    predictions: np.ndarray,
    score_func: callable,
    num_permutations: int = 1000,
    permutation_block_size: int = 10,
) -> Tuple[np.ndarray, np.ndarray]:
    """Compute the p-values of the given predictions using a permutation test.

    Parameters
    ----------
    targets : np.ndarray
        Ground truth.
    predictions : np.ndarray
        Predicted values.
    score_func : callable
        Callable function to compute the score.
    num_permutations : int, optional
        Number of permutations, by default 1000
    permutation_block_size : int, optional
        Block size, intended to keep correlation high, by default 10

    Returns
    -------
    pvalues : np.ndarray
        p-values.
    true_scores : np.ndarray
        True scores.
    """

    true_scores = score_func(targets, predictions)
    
    num_TRs = predictions.shape[0]
    blocks = np.array_split(np.arange(num_TRs), int(num_TRs / permutation_block_size))
    
    num_get_true_score = np.zeros(true_scores.shape)

    for permutation_num in tqdm(range(num_permutations)):
        _ = np.random.shuffle(blocks)
        permutation_order = np.concatenate(blocks)
        shuffled_pred = predictions[permutation_order]
        shuffled_scores = score_func(targets, shuffled_pred)
        num_get_true_score[shuffled_scores >= true_scores] += 1
    pvalues = num_get_true_score / num_permutations
    
    return pvalues, true_scores


In [ ]:
import multiprocessing

# parallel permutation test

def parallel_permutation_mp(
    targets: np.ndarray,
    predictions: np.ndarray,
    score_func: callable,
    num_permutations: int = 1000,
    permutation_block_size: int = 10,
    num_processes: int = 10,
) -> Tuple[np.ndarray, np.ndarray]:
    
    true_scores = score_func(targets, predictions)
    
    num_TRs = predictions.shape[0]
    blocks = np.array_split(np.arange(num_TRs), int(num_TRs / permutation_block_size))
        
    def single_test():
        num_get_true_score = np.zeros(true_scores.shape)    
        _ = np.random.shuffle(blocks)
        permutation_order = np.concatenate(blocks)
        shuffled_pred = predictions[permutation_order]
        shuffled_scores = score_func(targets, shuffled_pred)

        return num_get_true_score[shuffled_scores >= true_scores] + 1

    

In [17]:
bert_scores = trainer.get_scores()

In [18]:
test_selectivity = bert_scores["test_r2_selectivity"]

In [ ]:
plot_flatmap_from_mapper(
    test_selectivity,
    trainer.sub_config.sub_fmri_mapper_path,
    alpha=trainer.mask.astype(float),
    cmap="rainbow",
)

In [ ]:
trainer = Trainer(result_config_json=result_config_json_mbert["result_meta_file"])
prediction = trainer.refit_and_evaluate(return_pred=True)

In [ ]:
mbert_scores = trainer.get_scores() 
test_selectivity = mbert_scores["test_r2_selectivity"]
plot_flatmap_from_mapper(
    test_selectivity,
    trainer.sub_config.sub_fmri_mapper_path,
    alpha=trainer.mask.astype(float),
    cmap="rainbow",
)

# ARG MAX Timescale

In [ ]:
plot_timeline_flatmaps(
    result_config_json_mbert["result_meta_file"], feature_name="mBERT"
)

In [ ]:
plot_timeline_flatmaps(
    result_config_json_mbert["result_meta_file"], feature_name="mBERT", is_corr=False
)

In [ ]:
plot_timeline_flatmaps(result_config_json_bert["result_meta_file"], feature_name="BERT")

In [ ]:
plot_timeline_flatmaps(
    result_config_json_bert["result_meta_file"], feature_name="BERT", is_corr=False
)

In [ ]:
def plot_flatmap(
    train_config_paths: List[str],
    feature_config_paths: List[str],
    subject_config_paths: List[str],
    feature_index: int = 0,
    is_corr: bool = True,
    save_dir=".temp/plots/",
):
    configs = list(
        product(subject_config_paths, train_config_paths, feature_config_paths)
    )

    for subject_config_path, train_config_path, feature_config_path in configs:
        print(subject_config_path, train_config_path, feature_config_path)
        with open(subject_config_path, "r") as f:
            subject_config = SubjectConfig(**json.load(f))
        with open(train_config_path, "r") as f:
            train_config = TrainerConfig(**json.load(f))
        with open(feature_config_path, "r") as f:
            feature_config = FeatureConfig(**json.load(f))
        trainer = Trainer(subject_config, feature_config)
        trainer.plot(
            trainer_config=train_config,
            feature_index=feature_index,
            is_corr=is_corr,
        )

        if os.path.exists(save_dir) == False:
            os.makedirs(save_dir)

        save_path = os.path.join(
            save_dir,
            f"{subject_config.sub_id}_{subject_config.task}_{feature_config.timescale}.png",
        )

        # plt.title(
        #     f"{subject_config.sub_id}_{subject_config.task}_{feature_config.timescale}"
        # )
        plt.savefig(save_path)

# mBERT MOTEN

In [ ]:
# BERT
train_config_paths = [
    # ".temp/config/train/bert_trainer_config.json",
    ".temp/config/train/mbert_trainer_config_100.json"
]

feature_config_paths = os.listdir(".temp/config/feature/mbert")
feature_config_paths = [
    os.path.join(".temp/config/feature/mbert", path) for path in feature_config_paths
]

subject_config_paths = [
    ".temp/config/subject/subject-07-reading.json",
]

plot_flatmap(
    train_config_paths=train_config_paths,
    feature_config_paths=feature_config_paths,
    subject_config_paths=subject_config_paths,
    feature_index=-1,
    is_corr=True,
    save_dir=".temp/results/mbert/plots/",
)

# mBERT - timescale

In [ ]:
plot_flatmap(
    train_config_paths=train_config_paths,
    feature_config_paths=feature_config_paths,
    subject_config_paths=subject_config_paths,
    feature_index=0,
    is_corr=True,
    save_dir=".temp/results/mbert/plots/",
)

# BERT MOTEN

In [ ]:
# BERT
train_config_paths = [
    ".temp/config/train/bert_trainer_config.json",
]

feature_config_paths = os.listdir(".temp/config/feature/bert")
feature_config_paths = [
    os.path.join(".temp/config/feature/bert", path) for path in feature_config_paths
]

subject_config_paths = [
    ".temp/config/subject/subject-07-reading.json",
]

plot_flatmap(
    train_config_paths=train_config_paths,
    feature_config_paths=feature_config_paths,
    subject_config_paths=subject_config_paths,
    feature_index=-1,
    is_corr=True,
    save_dir=".temp/results/bert/plots/",
)

# BERT - timescale

In [ ]:
plot_flatmap(
    train_config_paths=train_config_paths,
    feature_config_paths=feature_config_paths,
    subject_config_paths=subject_config_paths,
    feature_index=0,
    is_corr=True,
    save_dir=".temp/results/bert/plots/",
)